In [13]:
from qibo.hamiltonians import SymbolicHamiltonian
from qibo import symbols
from double_bracket_evolution_oracles import *
from group_commutator_iteration_transpiler import *
from numpy.linalg import norm
"""Test create evolution oracle"""

h_x = SymbolicHamiltonian( symbols.X(0) + symbols.Z(0) * symbols.X(1) + symbols.Y(2) 
                              + symbols.Y(1) * symbols.Y(2), nqubits = 3 )
d_0 = SymbolicHamiltonian(symbols.Z(0), nqubits = 3 )
# Initialize with EvolutionOracleType hamiltonian_simulation so that d_0 is Delta
h_input = h_x + d_0
input_hamiltonian_evolution_oracle_hamiltonian_simulation = EvolutionOracle(deepcopy(h_input), "ZX",
                                                                mode_evolution_oracle = EvolutionOracleType.hamiltonian_simulation)

In [14]:
from double_bracket import *

dbi = DoubleBracketIteration(deepcopy(h_input.dense))


[Qibo 0.2.5|WARNING|2024-02-08 07:00:12]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.


In [15]:
t_step = 0.051#0.98#dbi.hyperopt_step()

In [16]:
dbi(t_step)
dbi.off_diagonal_norm

28.82580567725418

In [17]:
## Test more fancy functionalities
input_hamiltonian_evolution_oracle_hamiltonian_simulation.please_be_verbose = False
gci = GroupCommutatorIterationWithEvolutionOracles( deepcopy(input_hamiltonian_evolution_oracle_hamiltonian_simulation ))

In [18]:
u = gci.iterated_hamiltonian_evolution_oracle.circuit( np.sqrt(t_step)).unitary()
u2 = gci.iterated_hamiltonian_evolution_oracle.circuit( -np.sqrt(t_step)).unitary()
norm(u-u2.T.conj())

5.136025650750149e-16

In [19]:
d_0.mode_evolution_oracle = EvolutionOracleType.text_strings
gci.iterated_hamiltonian_evolution_oracle.mode_evolution_oracle = EvolutionOracleType.text_strings
query_list = gci.group_commutator( np.sqrt(t_step),
                                             diagonal_association_evolution_oracle= EvolutionOracle(d_0, "D0"))


query_list['forwards']

'ZX-0.22583179581272428D00.22583179581272428ZX0.22583179581272428D0-0.22583179581272428'

In [20]:

gci.iterated_hamiltonian_evolution_oracle.mode_evolution_oracle = EvolutionOracleType.numerical
query_list = gci.group_commutator( np.sqrt(t_step),
                                             diagonal_association_evolution_oracle= EvolutionOracle(d_0, "D0",mode_evolution_oracle=EvolutionOracleType.numerical))


norm(query_list['forwards'] -query_list['backwards'].T.conj())





[Qibo 0.2.5|WARNING|2024-02-08 07:00:13]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.
[Qibo 0.2.5|WARNING|2024-02-08 07:00:13]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.


7.193109780637881e-16

In [21]:
## Test more fancy functionalities
input_hamiltonian_evolution_oracle_hamiltonian_simulation.please_be_verbose = False
gci = GroupCommutatorIterationWithEvolutionOracles( deepcopy(input_hamiltonian_evolution_oracle_hamiltonian_simulation ))
d_ev =  EvolutionOracle(d_0, "D0",mode_evolution_oracle=EvolutionOracleType.hamiltonian_simulation)

gci.iterated_hamiltonian_evolution_oracle.mode_evolution_oracle = EvolutionOracleType.hamiltonian_simulation
query_list = gci.group_commutator( np.sqrt(t_step),
                                             diagonal_association_evolution_oracle= d_ev )

norm(query_list['forwards'].unitary() -query_list['backwards'].unitary().conj().T)




6.561656035791626e-16